# Project EdgeAI

## Import library to get data

In [27]:
import requests, os, shutil

listQuery = ["Emberiza citrinella", "Alauda arvensis", "Emberiza cirlus", "Cuculus canorus"] #, "Tyto alba", "Falco tinnunculus", "Sylvia borin", "Muscicapa striata", "Tachybaptus ruficollis", "Delichon urbicum"]

## Import data from API

In [28]:
url = "https://xeno-canto.org/api/2/recordings?query="

def enregistrer_mp3(url_api, nom_fichier):
    reponse = requests.get(url_api, stream=True)
    
    if reponse.status_code == 200:
        with open(nom_fichier, 'wb') as fichier_audio:
            reponse.raw.decode_content = True
            shutil.copyfileobj(reponse.raw, fichier_audio)
        print("Audio enregistré avec succès sous le nom :", nom_fichier)
    else:
        print("Erreur lors de la récupération des données audio depuis l'API")

def getData(query):
    url_api = url + query
    response = requests.get(url_api)

    if response.status_code == 200:
        contenu = response.json()["recordings"]
        pathToDir = "./data/" + query

        if not os.path.exists(pathToDir):
            os.mkdir(pathToDir)

            for _, element in zip(range(250), contenu):
                elementUrl = element["file"]
                elementFileName = element["file-name"]
                storeFile = pathToDir + "/" + elementFileName
                
                if not os.path.exists(storeFile):
                    try:
                        enregistrer_mp3(elementUrl, storeFile)
                    except: # Pour les problèmes de noms de fichiers
                        pass

for query in listQuery:
    getData(query)

## Import library to train model

In [29]:
from sklearn import model_selection
from pathlib import Path
import librosa
import copy
import matplotlib
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.activations import softmax

from keras.layers import Input, Conv1D, MaxPool1D, Flatten, Dense, AvgPool1D, Activation
from keras.utils import get_file
from keras.utils import to_categorical

dataset_dir = Path('data')

## Load all sound in X and Y

In [30]:
x = []
y = []

def slice_audio_with_trim(file_path, index, duration=1):
    audio_data, sampling_rate = librosa.load(file_path, sr=None)
    non_silent_audio, _ = librosa.effects.trim(audio_data)
    num_slices = int(len(non_silent_audio) / sampling_rate / duration)

    # Diviser l'audio en fonction de duration
    for i in range(num_slices):
        start = i * sampling_rate * duration
        end = start + sampling_rate * duration
        slice_data = non_silent_audio[start:end]

        data = slice_data.astype(np.float32)
        data.resize((16000, 1))

        x.append(data)
        y.append(index)

def getAllDataClasses(directory, index):
    for recording in dataset_dir.glob(f'{directory}/*'):
        slice_audio_with_trim(str(recording) ,index)

def loadAllData():
    for i in range(len(listQuery)):
        getAllDataClasses(listQuery[i], i)

loadAllData()

In [31]:
u, indexes, counts = np.unique(y, return_index=True, return_counts=True)
min = np.min(counts)
nbrClasses = len(u)

indices = np.random.permutation(min)

newX = []
newY = []

for i in range(nbrClasses):
    sousSetX = []
    sousSetY = []
    
    if i == nbrClasses - 1:
        sousSetX = x[indexes[i]:]
        sousSetY = y[indexes[i]:]
    else:
        sousSetX = x[indexes[i] : indexes[i + 1]]
        sousSetY = y[indexes[i] : indexes[i + 1]]
    
    x_selectionne = [sousSetX[j] for j in indices]
    y_selectionne = [sousSetY[j] for j in indices]

    for x_s, y_s in zip(x_selectionne, y_selectionne):
        newX.append(x_s)
        newY.append(y_s)

u, indexes, counts = np.unique(newY, return_index=True, return_counts=True)
print(u)
print(indexes)
print(counts)

[0 1 2 3]
[    0 12744 25488 38232]
[12744 12744 12744 12744]


In [32]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(newX, newY, test_size=0.2)
x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = to_categorical(np.array(y_train))
y_test = to_categorical(np.array(y_test))


In [33]:
print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)

(40780, 16000, 1) (10196, 16000, 1)
(40780, 4) (10196, 4)


In [34]:
model = Sequential()
model.add(Input(shape=(16000, 1)))
model.add(Conv1D(filters=4, kernel_size=3, activation='relu'))
model.add(MaxPool1D(pool_size=8))
model.add(Conv1D(filters=4, kernel_size=3, activation='relu'))
model.add(MaxPool1D(pool_size=2))
model.add(Flatten())
model.add(Dense(units=4))
model.add(Activation('softmax')) # SoftMax activation needs to be separate from Dense to remove it later on
# EXPLORE Learning Rate
opt = tf.keras.optimizers.Adam(learning_rate=10e-4)
model.summary()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 15998, 4)          16        
                                                                 
 max_pooling1d_2 (MaxPoolin  (None, 1999, 4)           0         
 g1D)                                                            
                                                                 
 conv1d_3 (Conv1D)           (None, 1997, 4)           52        
                                                                 
 max_pooling1d_3 (MaxPoolin  (None, 998, 4)            0         
 g1D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 3992)              0         
                                                                 
 dense_1 (Dense)             (None, 4)                

In [35]:
model.fit(x_train, y_train, epochs=50, batch_size=384, validation_data=(x_test, y_test))
# model.fit(x_train, y_train, epochs=25, batch_size=268, validation_data=(x_test, y_test))

Epoch 1/50
107/107 [==============================] - 28s 257ms/step - loss: 1.2438 - categorical_accuracy: 0.4627 - val_loss: 1.1615 - val_categorical_accuracy: 0.5154
Epoch 2/50
107/107 [==============================] - 26s 243ms/step - loss: 1.1374 - categorical_accuracy: 0.5111 - val_loss: 1.1222 - val_categorical_accuracy: 0.5278
Epoch 3/50
107/107 [==============================] - 26s 247ms/step - loss: 1.1077 - categorical_accuracy: 0.5215 - val_loss: 1.0970 - val_categorical_accuracy: 0.5376
Epoch 4/50
107/107 [==============================] - 26s 241ms/step - loss: 1.0800 - categorical_accuracy: 0.5363 - val_loss: 1.0760 - val_categorical_accuracy: 0.5381
Epoch 5/50
107/107 [==============================] - 26s 239ms/step - loss: 1.0611 - categorical_accuracy: 0.5456 - val_loss: 1.0598 - val_categorical_accuracy: 0.5389
Epoch 6/50
107/107 [==============================] - 25s 238ms/step - loss: 1.0487 - categorical_accuracy: 0.5525 - val_loss: 1.0503 - val_categorical_acc

In [36]:
model.evaluate(x_test, y_test, verbose=2)
pred_test = model.predict(x_test)
print(tf.math.confusion_matrix(y_test.argmax(axis=1), pred_test.argmax(axis=1)))

319/319 - 3s - loss: 0.9676 - categorical_accuracy: 0.5819 - 3s/epoch - 11ms/step
319/319 [==============================] - 4s 12ms/step
tf.Tensor(
[[1755  483  323   37]
 [ 409 1908  210   37]
 [1290  518  644   82]
 [ 298  195  381 1626]], shape=(4, 4), dtype=int32)


In [37]:
model.save('project.h5')
if isinstance(model.layers[-1], Activation) and model.layers[-1].activation == softmax:
    model = tf.keras.Model(model.input, model.layers[-2].output, name=model.name)
else:
    print('Error: last layer is not SoftMax Activation')

NameError: name 'softmax' is not defined

In [ ]:
%pip install qualia_codegen_core
import qualia_codegen_core
from qualia_codegen_core.graph.KerasModelGraph import KerasModelGraph
from qualia_codegen_core.graph.Quantization import Quantization
from qualia_codegen_core.graph.RoundMode import RoundMode

from importlib.resources import files
main_path = str((files('qualia_codegen_core.examples')/'Linux'/'main.cpp').resolve())


   ---------------------------------------- 0.0/4.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.5 MB 1.9 MB/s eta 0:00:03
   -- ------------------------------------- 0.3/4.5 MB 4.1 MB/s eta 0:00:02
   ----- ---------------------------------- 0.6/4.5 MB 5.4 MB/s eta 0:00:01
   -------- ------------------------------- 0.9/4.5 MB 5.8 MB/s eta 0:00:01
   ---------- ----------------------------- 1.2/4.5 MB 6.0 MB/s eta 0:00:01
   ------------- -------------------------- 1.5/4.5 MB 6.0 MB/s eta 0:00:01
   ---------------- ----------------------- 1.8/4.5 MB 6.1 MB/s eta 0:00:01
   ------------------ --------------------- 2.1/4.5 MB 6.1 MB/s eta 0:00:01
   --------------------- ------------------ 2.4/4.5 MB 6.2 MB/s eta 0:00:01
   ------------------------ --------------- 2.7/4.5 MB 6.2 MB/s eta 0:00:01
   --------------------------- ------------ 3.1/4.5 MB 6.3 MB/s eta 0:00:01
   ------------------------------ --------- 3.4/4.5 MB 6.4 MB/s eta 0:00:01
   ---------------


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip
Cannot find PyTorch, PyTorch framework will be unavailable


In [ ]:
modelgraph = KerasModelGraph(model).convert()
print(modelgraph)

—————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————
Inputs                                           | Layer                                            | Outputs                                          | Input shape                                      | Output shape                                    
—————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————
                                                 | input_1                                          | conv1d                                           | (1, 16000, 1)                                    | ((1, 16000, 1),)                   

In [ ]:
float_modelgraph = copy.deepcopy(modelgraph)

# layer quantization annotations for float32
for node in float_modelgraph.nodes:
    # No scale factor if not fixed-point quantization on integers
    node.q = Quantization(
            number_type=float,
            width=32,
            long_width=32,
            weights_scale_factor=0,
            output_scale_factor=0,
            weights_round_mode=RoundMode.NONE,
            output_round_mode=RoundMode.NONE,
            )

float_res = qualia_codegen_core.Converter(output_path=Path('gsc_output_floating')).convert_model(float_modelgraph)

with open('gsc_model_floating.h', 'w') as f:
    f.write(float_res)

Graphviz not available
Activation function TActivation.SOFTMAX not supported
ModelGraph validation failed


TypeError: write() argument must be str, not None

In [ ]:
!g++ -std=c++17 -Wall -Wextra -pedantic -Ofast -o gsc_fixed -include gsc_output_fixed/include/defines.h -Igsc_output_fixed/include gsc_output_fixed/model.c {main_path}

cc1plus.exe: fatal error: gsc_output_fixed/model.c: No such file or directory
compilation terminated.
<command-line>: fatal error: gsc_output_fixed/include/defines.h: No such file or directory
compilation terminated.
